# Team Members
# Tarik Koric netid:koric1
# Pranav Velamakanni netid: pranavv2

## Tarik: 
- mean and scale data 
- test data
- load the data and scale the orginal housing data

## Pranav:
- get intercepts and scale back
- test data
- set up python enviorment

In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [2]:
boston = load_boston()
Boston = pd.DataFrame(data = boston.data, columns=boston.feature_names)

In [3]:
Boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [4]:
X, y = load_boston(return_X_y=True)
iLog = [0,2,4,5,7,8,9]
for i in iLog:
    X[:,i] = np.log(X[:,i])
y = np.log(y)
X[:,1] = X[:,1]/10
X[:, 6] = np.power(X[:, 6],2.5) / 10000
X[:, 10] = np.exp(0.4 * X[:, 10]) / 1000
X[:, 11] = X[:, 11] / 100
X[:, 12] = np.sqrt(X[:, 12])
lam_seq = [0.30, 0.2, 0.1, 0.05, 0.02, 0.005]

In [5]:
def one_var_lasso(r, x, lam):
    xx = np.sum(x**2)
    xr = np.sum(r * x)
    b = (np.abs(xr) - lam/2)/xx
    if b > 0.0:
        b = b * np.sign(xr)
    else:
        b = 0.0
    return b

In [6]:
def MyLasso(X, y, lam_seq, maxit = 50):
    
    n = len(y)
    p = X.shape[1]
    nlam = len(lam_seq)
    
    
    #mean and scale
    y_mean = np.nanmean(y)
    
    
    X_mean = np.nanmean(X, axis=0)
    X_sd = np.nanstd(X, axis=0)
    
    
    Xs = np.zeros(shape = X.shape)
    for i in range(0,X.shape[1]):
        Xs[:,i] =  (X[:,i] - X_mean[i])/X_sd[i]
    
    yc = y - y_mean
    
    
    
     # Initilize coef vector b and residual vector r
    
    b = np.zeros(p)
    r = yc
    B = np.zeros(shape = (nlam,p+1))
    
    for m in range(0,nlam):
        lam = 2 * n * lam_seq[m]
        for step in range(0,maxit):
            for j in range(0,p):
                r = r + (Xs[:,j] * b[j])
                b[j] = one_var_lasso(r, Xs[:, j], lam)
                r = r - Xs[:, j] * b[j]
        B[m,1:] = b
    
    #get intercepts and scale coefficiants 
    

        
    for i in range(0,6):
        bjx =  0
        for j in range (1,14):
            bjx += B[i,j] * (X_mean[j-1]/X_sd[j-1])
        B[i,0] = y_mean - bjx
        
    for i in range(0,13):
        B[:,i+1] = B[:,i+1] /X_sd[i]
        

    return B
    

In [7]:
myout = MyLasso(X, y, lam_seq, maxit = 50)
myout = myout.transpose()


In [8]:
df = pd.read_csv('Coef_Lasso.dat')

In [9]:
df

,s0,s1,s2,s3,s4,s5
(Intercept),3.162393,3.508946,3.855936,3.778456,3.542129,3.925458
crim,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
zn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
indus,0.000000,0.000000,0.000000,0.000000,0.000000,-0.005399
chas,0.000000,0.000000,0.000000,0.000000,0.066692,0.099120
nox,0.000000,0.000000,0.000000,0.000000,0.000000,-0.125163
rm,0.000000,0.000000,0.000000,0.240416,0.417249,0.459273
age,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
dis,0.000000,0.000000,0.000000,0.000000,-0.004681,-0.120438
rad,0.000000,0.000000,0.000000,0.000000,0.000000,0.014980


In [10]:
df2 = pd.DataFrame(data=myout)
df2

,0,1,2,3,4,5
0,3.162393,3.508946,3.855935,3.777920,3.541573,3.927046
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,-0.005275
4,0.000000,0.000000,0.000000,0.000000,0.066670,0.099049
5,0.000000,0.000000,0.000000,0.000000,0.000000,-0.123495
6,0.000000,0.000000,0.000000,0.240724,0.417614,0.459680
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,-0.004686,-0.119808
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.015119


In [11]:
lasso_fit = df.to_numpy()

In [12]:
np.max(np.abs(lasso_fit - myout))

0.0016676950478649588